In [1]:
#pip install sentence-transformers

In [2]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import networkx as nx
import json
import langid
from sklearn.feature_extraction.text import TfidfVectorizer
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
from nltk import pos_tag, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from collections import Counter
import ast

# Download the NLTK data for part-of-speech tagging
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')
from nltk.corpus import words

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vandi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vandi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\vandi\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)

# Creating the Dataset

## Creating CSV from JSON

In [4]:
json_file=pd.read_json('sm_pads.json', orient='records')

In [5]:
pads_list = json_file['pads']
flattened_data = []

for pad in pads_list:
    flattened_pad = {
        'id': pad['id'],
        'content': pad['content'],
        'is_public': pad['is_public'],
        'tags': '; '.join(pad['tags']),  # Join the list of tags into a string
        'sdgs': '; '.join(pad['sdgs']),  # Join the list of sdgs into a string
        'country': pad['country'],
        'lang': pad['lang']
    }
    flattened_data.append(flattened_pad)

In [6]:
df = pd.DataFrame(flattened_data)
df.head()

,id,content,is_public,tags,sdgs,country,lang
0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,clean energy; affordable energy; stove,Good health and well-being; Affordable and cle...,ETH,en
1,4571,Alokito hridoy - enlightening the souls of chi...,True,education,Quality education,BGD,en
2,6099,BioFabricating Materials\nSolution holder: Bat...,False,nature-based solutions; design; sustainable fa...,"Industry, innovation and infrastructure; Respo...",JOR,en
3,6100,Circulamic\nSolution holder: Batoul Al-Rashdan...,False,nature-based solutions; recycling; design; sus...,"Industry, innovation and infrastructure; Respo...",JOR,en
4,4145,Gakyid Ride (City Bus ride app)\nThis app prov...,False,digital platform; innovation challenges; innov...,Good health and well-being; Sustainable cities...,BTN,en


In [7]:
df.dropna(inplace=True)
df.shape
df['tags'] = df['tags'].str.split('; ')
df_explode=df.explode('tags')
tag_distribution = df_explode['tags'].value_counts()
#df['tags'] = df['tags'].apply(ast.literal_eval)

In [8]:
#df_explode = df.explode('tags')

In [9]:
print(df['tags'][0])

['clean energy', 'affordable energy', 'stove']


In [10]:
df_explode.head()

,id,content,is_public,tags,sdgs,country,lang
0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,clean energy,Good health and well-being; Affordable and cle...,ETH,en
0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,affordable energy,Good health and well-being; Affordable and cle...,ETH,en
0,4364,Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...,True,stove,Good health and well-being; Affordable and cle...,ETH,en
1,4571,Alokito hridoy - enlightening the souls of chi...,True,education,Quality education,BGD,en
2,6099,BioFabricating Materials\nSolution holder: Bat...,False,nature-based solutions,"Industry, innovation and infrastructure; Respo...",JOR,en


In [11]:
df_explode['unique_id'] = range(len(df_explode))
df_explode = df_explode[['unique_id'] + [col for col in df_explode.columns if col != 'unique_id']]
print(df_explode[df_explode['tags'] == 'clean energy'].iloc[0])

unique_id                                                    0
id                                                        4364
content      Tikikil stove\n\nGIZ\nOffice contact GIZ Offic...
is_public                                                 True
tags                                              clean energy
sdgs         Good health and well-being; Affordable and cle...
country                                                    ETH
lang                                                        en
Name: 0, dtype: object


In [12]:
df.to_csv("capstone_data_final.csv")

## Extract Tags from the Content

In [13]:
def remove_non_english_tokens(text):
    '''Remove all non english tokens'''
    # Split the text into tokens

    # Identify the language of each token
    english_tokens = [word for word in text.split() if langid.classify(word)[0] == 'en']

    # Reconstruct the text with English tokens
    filtered_text = ' '.join(english_tokens)

    return filtered_text

In [14]:
def remove_proper_nouns(text):
    '''remove proper nouns such as names of persons'''
    # Tokenize the text into words
    words = word_tokenize(text)

    # Perform part-of-speech tagging
    pos_tags = pos_tag(words)

    # Remove proper nouns (NNP: singular proper noun, NNPS: plural proper noun)
    filtered_words = [word for word, pos in pos_tags if pos not in ['NNP', 'NNPS']]

    # Reconstruct the text without proper nouns
    filtered_text = ' '.join(filtered_words)

    return filtered_text

In [15]:
def remove_emails_and_hyperlinks(text):
    '''remove emails and hyperlinks'''
    # Remove emails
    text_no_emails = re.sub(r'\S+@\S+', '', text)

    # Remove hyperlinks
    
    text_no_links = re.sub(r'http[s]?\S+', '', text_no_emails,flags=re.IGNORECASE)

    return text_no_links

In [16]:
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

In [17]:
df=pd.read_csv("capstone_data_final.csv")
df=df[df['lang']=='en']

In [ ]:
df["clean_content"]=df["content"].apply(remove_non_english_tokens)
df["clean_content"]=df["clean_content"].apply(remove_emails_and_hyperlinks)
df["clean_content"]=df["clean_content"].apply(remove_proper_nouns)
df["clean_content"]=df["clean_content"].apply(remove_numbers)

In [ ]:
model = KeyBERT(model="distilbert-base-nli-mean-tokens")

# Function to extract keywords (tags) for each text
def extract_tags(text, top_n=5,keyprob=False):
    '''extract top keywords based on the parameter given by user'''
    keywords = model.extract_keywords(text,top_n=top_n)
    return [keyword[0] for keyword in keywords if keyword[1]>=0.1]

# Apply the extract_tags function to each text in the DataFrame
df["tags_keybert"] = df["clean_content"].apply(lambda text: extract_tags(text, top_n=20,keyprob=True))

In [ ]:
df.to_csv("keybert_thresh_onlyen_0.1_nltk.csv")

##  Semantic clustering of the tags (user assigned + KeyBERT) using SBERT + Root word generation loop

In [ ]:
def cluster_creation(combined_df):
    combined_df_explode = combined_df.explode('combined_tags')
    combined_df_explode['unique_id'] = range(len(combined_df_explode))
    combined_df_explode = combined_df_explode[['unique_id'] + [col for col in combined_df_explode.columns if col != 'unique_id']]
    combined_df_explode.set_index('unique_id', inplace=True)
    combined_df_explode = combined_df_explode.applymap(lambda s: s.lower() if type(s) == str else s)
    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    combined_corpus = combined_df_explode['combined_tags'].unique()
    combined_corpus = list(combined_corpus)
    combined_corpus_embeddings = embedder.encode(combined_corpus, show_progress_bar=True)
    combined_corpus_embeddings = combined_corpus_embeddings /  np.linalg.norm(combined_corpus_embeddings, axis=1, keepdims=True)
    combined_clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold = 0.75, metric='cosine', linkage='average')
    combined_clustering_model.fit(combined_corpus_embeddings)
    combined_cluster_assignment = combined_clustering_model.labels_
    combined_clustered_sentences = {}
    for sentence_id, cluster_id in enumerate(combined_cluster_assignment):
        if cluster_id not in combined_clustered_sentences:
            combined_clustered_sentences[cluster_id] = []

        combined_clustered_sentences[cluster_id].append(combined_corpus[sentence_id])
    #print(combined_clustered_sentences)
    sbert_clusters = {key: [str(value).strip("'") for value in values] for key, values in combined_clustered_sentences.items()}
    return sbert_clusters

In [ ]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

nlp = spacy.load("en_core_web_md")
def central_word_cossim(row):
    # Calculate aggregated vector
    #vectors = [nlp(word).vector for word in words]
    
    ####Normalize first before mean, then cos_sim not required
    
    
    aggregated_vector = sum(nlp(word).vector/len(nlp(word).vector) for word in row) / len(row)
    #print(aggregated_vector)
    
    #Reshape to 2-D
    #word = word.reshape(1, -1)
    aggregated_vector = aggregated_vector.reshape(1, -1)
    
    
    # Calculate similarity between aggregated vector and each word vector
    similarities = [cosine_similarity(aggregated_vector, nlp(word).vector.reshape(1,-1)) for word in row]
    

    # Select the word with the highest similarity
    central_word_index = np.argmax(similarities)
    return row[central_word_index]

#cluster_df["keywords_glove_lemm"]=cluster_df['tags_lemmatized'].apply(lambda row: central_word_cossim(row))

In [ ]:
def root_word_gen(sbert_clusters):
    #print(sbert_clusters)
    dict_df={"cluster":sbert_clusters.keys(),"tags":sbert_clusters.values()}
    tags_cluster=[]
    for i in dict_df["tags"]:
        tags=[]
        for j in i:
            if " " in j:
                j=j.split()
                for k in j:
                    tags.append(k)
            else:
                tags.append(j)
        tags_cluster.append(tags)
    cluster_df=pd.DataFrame.from_dict(dict_df)
    cluster_df["keywords_glove"]=cluster_df['tags'].apply(lambda row: central_word_cossim(row))
    cluster_dict = {}
    for i,data in cluster_df.iterrows():
        cluster_dict[data['keywords_glove']] = data['tags']
    #print(cluster_dict)
    
    return cluster_dict

In [ ]:
def map_tags_to_clusters(tag_list):
    clusters = []
    for key, value in cluster_dict.items():
        for tag in tag_list:
            if tag in value:
                clusters.append(key)
        
    #clusters = [key for key, value in sbert_clusters.items() if any(tag in value for tag in tag_list)]
    return clusters


In [ ]:
df_orig = pd.read_csv("capstone_data_final.csv")

In [ ]:
num_clusters = 100
combined_df = pd.read_csv("keybert_thresh_onlyen_0.1_nltk.csv")
combined_df['tags'] = combined_df['tags'].apply(ast.literal_eval)
combined_df['tags_keybert'] = combined_df['tags_keybert'].apply(ast.literal_eval)
combined_df['combined_tags'] = combined_df['tags'] + combined_df['tags_keybert']
for index, row in combined_df.iterrows():
    tags_list = row['tags'].copy()
    tags_keybert_list = row['tags_keybert'].copy()
    if len(tags_list) < 5:
        for word in tags_keybert_list:
            if word not in tags_list and len(tags_list)<5 and word.lower() in words.words():
                tags_list.append(word)
        combined_df.at[index, 'combined_tags'] = tags_list
#combined_df['combined_tags'] = combined_df['tags'] + combined_df['tags_keybert']
df_orig_loop = df_orig.copy()
#df_orig_loop.to_csv("clusters_master_df.csv")
num = 1
while num_clusters > 50:
    sbert_clusters = cluster_creation(combined_df)
    num_clusters = len(sbert_clusters.keys())
    print("clusters created")
    print(num_clusters)
    cluster_dict = root_word_gen(sbert_clusters)
    print("generated root word")
    df_orig = combined_df.copy()
    df_orig['tag_clusters'] = df_orig['combined_tags'].apply(map_tags_to_clusters)
    print('tag clusters mapped')
    df_orig.to_csv("clusters_master_df_"+str(num)+".csv")
    num+=1
    combined_df = df_orig[["tag_clusters"]]
    combined_df = combined_df.rename(columns={"tag_clusters": "combined_tags"})
    print('new df created')

In [ ]:
left = pd.read_csv("clusters_master_df_"+str(1)+".csv")
for i in range(1,num-1):
    right = pd.read_csv("clusters_master_df_"+str(i+1)+".csv")
    df_cluster = left.merge(right, how = "inner", left_on="tag_clusters", right_on="combined_tags")
    left = df_cluster.drop(["tag_clusters_x","combined_tags_y"],axis=1)
    left = left.rename({"tag_clusters_y":"tag_clusters"},axis=1)

In [ ]:
left

## Co-occurence of Clusters

In [ ]:
correlation_matrix = np.zeros((num_clusters, num_clusters), dtype=int)
document_counts = np.zeros(num_clusters, dtype=int)
clusters = list(cluster_dict.keys())
left['tag_clusters'] = left['tag_clusters'].apply(ast.literal_eval)
for tags in left['tag_clusters']:
    #print(tags)
    for tag in tags:
        cluster_index = clusters.index(tag)
        document_counts[cluster_index] += 1
    
    # Update the matrix based on the co-occurrence of clusters in each row
    for i in range(len(tags)):
        for j in range(i + 1, len(tags)):
            # Find the indices of the clusters in the matrix
            index_i = clusters.index(tags[i])
            index_j = clusters.index(tags[j])
            
            if index_i == index_j:
                continue

            # Update the matrix
            correlation_matrix[index_i, index_j] += 1
            correlation_matrix[index_j, index_i] += 1

# Create a DataFrame from the matrix
correlation_df = pd.DataFrame(correlation_matrix, columns=clusters, index=clusters)

# Normalize the correlation_df
for i in range(len(correlation_df.index)):
    for j in range(i + 1, len(correlation_df.columns)):
        denominator = document_counts[i] + document_counts[j]
        if denominator > 0:
            correlation_df.iloc[i, j] /= denominator
            correlation_df.iloc[j, i] /= denominator

print("Correlation Matrix:")
correlation_df

In [ ]:
figure = plt.figure(figsize=(10,10))
sns.heatmap(correlation_df, cmap="coolwarm",annot=False, linewidths=0)
plt.show()